In [1]:
import os
import glob
import datetime

import numpy as np
import scipy
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt

import obspy
from obspy.signal import PPSD


RAW_DATA_HOME = './rawdata'
RAW_BEARING_DATA_HOME = os.path.join(RAW_DATA_HOME, 'bearing')
RAW_TARGET_DATA_HOME = '2nd_test/2nd_test'
RAW_TARGET_DATA_PATTERN = '*'

RAW_DATA_FIND_PATTERN = os.path.join(RAW_BEARING_DATA_HOME,
                                os.path.join(RAW_TARGET_DATA_HOME, RAW_TARGET_DATA_PATTERN))

## Preparing data for 1st_test data
Each data set consists of individual files that are 1-second vibration signal snapshots recorded at specific intervals. For dataset 1, First 43 snapshots excluded from inspection.

In [2]:
# Grap files using pattern
streams = [obspy.core.stream.Stream() for _ in range(4)]

conv_targets = glob.glob(RAW_DATA_FIND_PATTERN)
conv_targets = sorted(conv_targets)
# Excluding first 43 samples.
for target in conv_targets:
    fname = os.path.basename(target)
    # get recording time from file name
    tokens = list(map(int, fname.strip().split('.')))
    time = datetime.datetime(*tokens)
    df = pd.read_csv(target, sep='\t')
    df.columns = ['b1', 'b2', 'b3', 'b4']
    for i, (name, values) in enumerate(df.iteritems()):
        # print(i, name, values.describe())
        stats = {
            'sampling_rate': 20480,
            'station': name.split('_')[0],
            'channel': 'acc',
            'starttime': time
        }
        v = values.values
        v -= np.mean(v)
        trace = obspy.core.trace.Trace(header=stats, data=v)
        # trace.plot()
        # print(i, stats)
        streams[i].append(trace)

## Inspect signal using PDF
Sampling rate of each dataset is 20,480. To 

In [3]:
ppsds = []
paz = {
    'gain': 1,
    'sensitivity': 1,
    'poles': [],
    'zeros': [],
}

for s in streams:
    s[0].stats.sampling_rate = 20479
    ppsd = PPSD(s[0].stats, paz, skip_on_gaps=True, ppsd_length=1, db_bins=(-75, -40, .1), special_handling='ringlaser')
    for t in s:
        t.stats.sampling_rate = 20479
        if not ppsd.add(t):
            print('TT')
    ppsds.append(ppsd)

In [6]:
for ppsd in ppsds:
    ppsd.plot(cmap=obspy.imaging.cm.pqlx, xaxis_frequency=True, show_coverage=False, show_noise_models=False, period_lim=(5, 10240)
              , filename=f'outs/dataset2/{ppsd.station}{ppsd.channel}')

/Users/jslee/anaconda/anaconda3/lib/python3.8/site-packages/obspy/signal/spectral_estimation.py:2028: UserWarning: Attempted to set non-positive right xlim on a log-scaled axis.
Invalid limit will be ignored.
  ax.invert_xaxis()
/Users/jslee/anaconda/anaconda3/lib/python3.8/site-packages/obspy/signal/spectral_estimation.py:2028: UserWarning: Attempting to set identical left == right == 1.0 results in singular transformations; automatically expanding.
  ax.invert_xaxis()


In [8]:
st = obspy.core.utcdatetime.UTCDateTime('2004-02-12T11:02:39')
mt = obspy.core.utcdatetime.UTCDateTime('2004-02-13T23:52:39') # Take snapshots 10 days long from start
et = obspy.core.utcdatetime.UTCDateTime('2004-12-19')

## Apply PCA to reduce feature dimention

In [11]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import LocalOutlierFactor
from sklearn.manifold import TSNE
from matplotlib.lines import Line2D # For mock element

colors = np.array(['r', 'b'])

preds = {}

for ppsd in ppsds:
    idx = 0
    for i, tp in enumerate(ppsd.times_processed):
        if tp >= mt:
            idx = i
            break
    X_train, X_test = ppsd.psd_values[:i], ppsd.psd_values[i:]
    X_train, X_test = np.asarray(X_train), np.asarray(X_test)
    print(X_train.shape, X_test.shape)
    scaler = StandardScaler().fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    
    pca = PCA(n_components=10)
    Y_train = pca.fit_transform(X_train_scaled)
    
    X_test_scaled = scaler.transform(X_test)
    Y_test = pca.transform(X_test_scaled)
    
    # Training LOF
    # Magic number taken from example
    clf = LocalOutlierFactor(n_neighbors=20, contamination=0.002, novelty=True)
    clf.fit(X_train_scaled)
    
    y_pred_result = clf.predict(X_test_scaled)
    # print(y_pred_result)
    preds[f'{ppsd.station}{ppsd.channel}'] = y_pred_result
    
    
    # Training t-sne model to map 10-dim to 2-dim
    tsne = TSNE(n_components=2, init='random', random_state=1231)
    
    X_tsne = np.vstack((Y_train, Y_test))
    Y_tsne = tsne.fit_transform(X_tsne)
    
    
    plt.scatter(Y_tsne[:i, 0], Y_tsne[:i, 1], color='k', alpha=0.3, zorder=1)
    plt.scatter(Y_tsne[i:, 0], Y_tsne[i:, 1], color=colors[(y_pred_result + 1) // 2], alpha=0.3, zorder=0)
    plt.title(f'{ppsd.station}, {ppsd.channel}')
    
    legend_elements = [
        Line2D([], [], color='k', linestyle='None', marker='o', label='Training data'),
        Line2D([], [], color='b', linestyle='None', marker='o', label='Test data, inlier'),
        Line2D([], [], color='r', linestyle='None', marker='o', label='Test data, outlier')
    ]
    
    plt.legend(handles=legend_elements)
     # plt.show()
    plt.savefig(f'./outs/dataset2/{ppsd.station}{ppsd.channel}_tsne.png', bbox_inches='tight')
    plt.close()
    
    
    

(224, 89) (760, 89)
(224, 89) (760, 89)
(224, 89) (760, 89)
(224, 89) (760, 89)


In [13]:
for stream in streams:
    msee = []
    c = []
    labels = preds[f'{stream[0].stats.station}{stream[0].stats.channel}']
    train_raw = stream.slice(st, mt)
    test_raw = stream.slice(mt, et)
    for t in train_raw:
        mse = np.sqrt(np.mean(t.data ** 2))
        msee.append(mse)
        c.append('k')
    for i, t in enumerate(test_raw):
        mse = np.sqrt(np.mean(t.data ** 2))
        msee.append(mse)
        c.append(colors[(labels[i] + 1) // 2])
    
    xaxis = np.arange(len(msee))
    plt.scatter(xaxis, msee, color=c, s=2)
    legend_elements = [
        Line2D([], [], color='k', linestyle='None', marker='o', label='Training data'),
        Line2D([], [], color='b', linestyle='None', marker='o', label='Test data, inlier'),
        Line2D([], [], color='r', linestyle='None', marker='o', label='Test data, outlier')
    ]
    plt.legend(handles=legend_elements)
    plt.title(f'{stream[0].stats.station}, {stream[0].stats.channel}')
    plt.ylabel('RMS')
    plt.xlabel('Snapshot')
    # plt.show()
    plt.savefig(f'./outs/dataset2/{stream[0].stats.station}{stream[0].stats.channel}_rms.png', bbox_inches='tight')
    plt.close()